## 결론부터

fastai 라이브러리를 이용, 데이터를 전처리 후 각각 random forest 와 GridsearchCV로 prediction

=> random forest의 성능이 더 나았다.  (RF : 100 초반,  GridsearchCV : 200 초반)

=> 리더보드에선 하위권이다 ㅜㅜ.. 그러나 fastai의 기능을 활용해보는 것에 의의를 두자.

=> feature importance를 살펴보면 GridsearchCV의 경우 특정 feature의 영향을 너무 높게 받는데, 이러한 이유로 인해 오히려 성능이 더 안좋아진 것으로 보인다.

=> 2021년도부터는 재택근무자 수가 늘어났는데, 과거데이터는 재택근무자 수가 거의 없다보니 재택근무자 수가 늘어났을 때의 수요변화의 영향을 덜 받는 것 같다. 재택근무자 수의 feature importance를 강제로 늘리거나, 재택근무자 수가 아예없는 과거데이터를 잘라내는 것도 score를 높이는 하나의 방법일 수 있다. 그러나 해당 데이터의 사이즈가 너무 작아 이 부분을 고려할 필요가 있다.

In [1]:
# 필요한 패키지들을 가져온다.
import pandas as pd
import numpy as np
import fastai
from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display
from sklearn import metrics
import gc
import math
# 이번 대회의 평가기준은 mae
from sklearn import metrics
from sklearn.metrics import mean_absolute_error as mae
from fastai.imports import *
from fastai.tabular import *

In [2]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [3]:
path = '../cafeteria_prediction/data/'
df = pd.read_csv(f'{path}train.csv')
test = pd.read_csv(f'{path}test.csv')
submission = pd.read_csv(f'{path}sample_submission.csv')
df.head(1)

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴,중식계,석식계
0,2016-02-01,월,2601,50,150,238,0.0,"모닝롤/찐빵 우유/두유/주스 계란후라이 호두죽/쌀밥 (쌀:국내산) 된장찌개 쥐어채무침 포기김치 (배추,고추가루:국내산)","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 오징어찌개 쇠불고기 (쇠고기:호주산) 계란찜 청포묵무침 요구르트 포기김치 (배추,고추가루:국내산)","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 육개장 자반고등어구이 두부조림 건파래무침 포기김치 (김치:국내산)",1039.0,331.0


In [9]:
test['중식계'] = np.nan
test['석식계'] = np.nan

df_full = df.append(test)

df_full.isna().sum()

일자                 0
요일                 0
본사정원수              0
본사휴가자수             0
본사출장자수             0
본사시간외근무명령서승인건수     0
현본사소속재택근무자수        0
조식메뉴               0
중식메뉴               0
석식메뉴               0
중식계               50
석식계               50
dtype: int64

In [10]:
test.shape

(50, 12)

In [4]:
# fastai old version(0.7.0)에 있는 함수들을 사용하기 위해 따로 함수 지정해준다.
# proc_df : 변수들을 수치화해준다. 결측치는 median으로 채워준다.

def numericalize(df, col, name, max_n_cat):
    """ Changes the column col from a categorical type to it's integer codes.
    """
    if not is_numeric_dtype(col) and ( max_n_cat is None or len(col.cat.categories)>max_n_cat):
        df[name] = pd.Categorical(col).codes+1

def fix_missing(df, col, name, na_dict):
    """ Fill missing data in a column of df with the median, and add a {name}_na column
    which specifies if the data was missing.
    """
    if is_numeric_dtype(col):
        if pd.isnull(col).sum() or (name in na_dict):
            df[name+'_na'] = pd.isnull(col)
            filler = na_dict[name] if name in na_dict else col.median()
            df[name] = col.fillna(filler)
            na_dict[name] = filler
    return na_dict

def proc_df(df, y_fld=None, skip_flds=None, ignore_flds=None, do_scale=False, na_dict=None,
            preproc_fn=None, max_n_cat=None, subset=None, mapper=None):
    """ proc_df takes a data frame df and splits off the response variable, and
    changes the df into an entirely numeric dataframe.

    Parameters:
    -----------
    df: The data frame you wish to process.

    y_fld: The name of the response variable

    skip_flds: A list of fields that dropped from df.

    ignore_flds: A list of fields that are ignored during processing.

    do_scale: Standardizes each column in df. Takes Boolean Values(True,False)

    na_dict: a dictionary of na columns to add. Na columns are also added if there
        are any missing values.

    preproc_fn: A function that gets applied to df.

    max_n_cat: The maximum number of categories to break into dummy values, instead
        of integer codes.

    subset: Takes a random subset of size subset from df.

    mapper: If do_scale is set as True, the mapper variable
        calculates the values used for scaling of variables during training time (mean and standard deviation).

    Returns:
    --------
    [x, y, nas, mapper(optional)]:

        x: x is the transformed version of df. x will not have the response variable
            and is entirely numeric.

        y: y is the response variable

        nas: returns a dictionary of which nas it created, and the associated median.

        mapper: A DataFrameMapper which stores the mean and standard deviation of the corresponding continuous
        variables which is then used for scaling of during test-time.

    Examples:
    ---------
    >>> df = pd.DataFrame({'col1' : [1, 2, 3], 'col2' : ['a', 'b', 'a']})
    >>> df
       col1 col2
    0     1    a
    1     2    b
    2     3    a

    note the type of col2 is string

    >>> train_cats(df)
    >>> df

       col1 col2
    0     1    a
    1     2    b
    2     3    a

    now the type of col2 is category { a : 1, b : 2}

    >>> x, y, nas = proc_df(df, 'col1')
    >>> x

       col2
    0     1
    1     2
    2     1

    >>> data = DataFrame(pet=["cat", "dog", "dog", "fish", "cat", "dog", "cat", "fish"],
                 children=[4., 6, 3, 3, 2, 3, 5, 4],
                 salary=[90, 24, 44, 27, 32, 59, 36, 27])

    >>> mapper = DataFrameMapper([(:pet, LabelBinarizer()),
                          ([:children], StandardScaler())])

    >>>round(fit_transform!(mapper, copy(data)), 2)

    8x4 Array{Float64,2}:
    1.0  0.0  0.0   0.21
    0.0  1.0  0.0   1.88
    0.0  1.0  0.0  -0.63
    0.0  0.0  1.0  -0.63
    1.0  0.0  0.0  -1.46
    0.0  1.0  0.0  -0.63
    1.0  0.0  0.0   1.04
    0.0  0.0  1.0   0.21
    """
    if not ignore_flds: ignore_flds=[]
    if not skip_flds: skip_flds=[]
    if subset: df = get_sample(df,subset)
    ignored_flds = df.loc[:, ignore_flds]
    df.drop(ignore_flds, axis=1, inplace=True)
    df = df.copy()
    if preproc_fn: preproc_fn(df)
    if y_fld is None: y = None
    else:
        if not is_numeric_dtype(df[y_fld]): df[y_fld] = df[y_fld].cat.codes
        y = df[y_fld].values
        skip_flds += [y_fld]
    df.drop(skip_flds, axis=1, inplace=True)

    if na_dict is None: na_dict = {}
    for n,c in df.items(): na_dict = fix_missing(df, c, n, na_dict)
    if do_scale: mapper = scale_vars(df, mapper)
    for n,c in df.items(): numericalize(df, c, n, max_n_cat)
    df = pd.get_dummies(df, dummy_na=True)
    df = pd.concat([ignored_flds, df], axis=1)
    res = [df, y, na_dict]
    if do_scale: res = res + [mapper]
    return res

In [5]:
# model fitting 후 score확인 함수
def print_score(m, X_train, y_train, X_valid, y_valid):
    res = [mae(m.predict(X_train), y_train), mae(m.predict(X_valid), y_valid),
                m.score(X_train, y_train), m.score(X_valid, y_valid)]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print(res)

In [11]:
# 자기개발 등등 석식계 0 인 날 처리 (제각각으로 씌여진 문구 하나로 통일하기)
df_full.loc[(df_full['석식계'] < 1)&(df_full['석식메뉴'].str.contains('\*[^ㄱ-힣]')),'석식메뉴'] = '*'
df_full.loc[(df_full['석식메뉴'].str.contains('자기개발|자기계발|자기 개발|자기 계발')),'석식메뉴'] = '자기계발의날'
df_full.loc[(df_full['석식메뉴'].str.contains('가정의')),'석식메뉴'] = '가정의날'

In [12]:
## feature engineering
def make_date(df, date_field):
    "Make sure `df[date_field]` is of the right date type."
    field_dtype = df[date_field].dtype
    if isinstance(field_dtype, pd.core.dtypes.dtypes.DatetimeTZDtype):
        field_dtype = np.datetime64
    if not np.issubdtype(field_dtype, np.datetime64):
        df[date_field] = pd.to_datetime(df[date_field], infer_datetime_format=True)
        
def add_datepart(df, field_name, prefix=None, drop=True, time=False):
    "Helper function that adds columns relevant to a date in the column `field_name` of `df`."
    make_date(df, field_name)
    field = df[field_name]
    prefix = ifnone(prefix, re.sub('[Dd]ate$', '', field_name))
    attr = ['Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear', 'Is_month_end', 'Is_month_start',
            'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start']
    if time: attr = attr + ['Hour', 'Minute', 'Second']
    # Pandas removed `dt.week` in v1.1.10
    week = field.dt.isocalendar().week.astype(field.dt.day.dtype) if hasattr(field.dt, 'isocalendar') else field.dt.week
    for n in attr: df[prefix + n] = getattr(field.dt, n.lower()) if n != 'Week' else week
    mask = ~field.isna()
    df[prefix + 'Elapsed'] = np.where(mask,field.values.astype(np.int64) // 10 ** 9,np.nan)
    if drop: df.drop(field_name, axis=1, inplace=True)
    return df

In [13]:
# 일자를 여러형태로 만들기
add_datepart(df_full, '일자', drop=False)

display(df_full.tail(1).T)

,49
일자,2021-04-09 00:00:00
요일,금
본사정원수,2973
본사휴가자수,259
본사출장자수,268
본사시간외근무명령서승인건수,0
현본사소속재택근무자수,229.0
조식메뉴,모닝롤/토마토샌드 우유/주스 계란후라이 채소죽/흑미밥 대구지리 애호박나물볶음 양상추샐러드 포기김치
중식메뉴,쌀밥/흑미밥/찰현미밥 사골우거지국 해물누룽지탕 청포묵*양념간장 비름나물고추장무침 석박지 그린샐러드*망고D
석식메뉴,흑미밥 맑은버섯국 매운사태조림 춘권*타르타르D 열무나물무침 포기김치


In [15]:
# 순수 본사근무정원수 추가하기
df_full = df_full.assign(net_work_num = df_full['본사정원수'] - (df_full['본사휴가자수'] + df_full['본사출장자수']))
df_full = df_full.assign(net_office_num = df_full['본사정원수'] - (df_full['본사휴가자수'] + df_full['본사출장자수'] + df_full['현본사소속재택근무자수']))
df_full = df_full.assign(off_office_num = df_full['본사휴가자수'] + df_full['본사출장자수'] + df_full['현본사소속재택근무자수'])

df_full = df_full.assign(net_office_ratio = df_full['net_office_num'] / df_full['본사정원수'])
df_full = df_full.assign(off_office_ratio = df_full['off_office_num'] / df_full['본사정원수'])

print(df_full[df_full['net_office_num'] <= df_full['중식계']].shape)

df_full = df_full.assign(overtime_work_ratio = df_full['본사시간외근무명령서승인건수']/df_full['net_work_num'])

#plt.scatter(df['본사시간외근무명령서승인건수'], df['석식계'])

# 석식메뉴 존재여부 표기
df_full['dinner_isnull'] = False
df_full.loc[df_full['석식메뉴'].isin(['*','자기계발의날', '가정의날']), ['dinner_isnull']] = True

df_full = df_full.assign(work_home_ratio = df_full['현본사소속재택근무자수'] / df_full['net_work_num'])

df_raw = df_full.drop(columns = ['조식메뉴','중식메뉴','석식메뉴'])

(0, 30)


In [16]:
df_raw.columns

Index(['일자', '요일', '본사정원수', '본사휴가자수', '본사출장자수', '본사시간외근무명령서승인건수',
       '현본사소속재택근무자수', '중식계', '석식계', '일자Year', '일자Month', '일자Week', '일자Day',
       '일자Dayofweek', '일자Dayofyear', '일자Is_month_end', '일자Is_month_start',
       '일자Is_quarter_end', '일자Is_quarter_start', '일자Is_year_end',
       '일자Is_year_start', '일자Elapsed', 'net_work_num', 'net_office_num',
       'off_office_num', 'net_office_ratio', 'off_office_ratio',
       'overtime_work_ratio', 'dinner_isnull', 'work_home_ratio'],
      dtype='object')

In [17]:
df_raw.isna().sum()

일자                      0
요일                      0
본사정원수                   0
본사휴가자수                  0
본사출장자수                  0
본사시간외근무명령서승인건수          0
현본사소속재택근무자수             0
중식계                    50
석식계                    50
일자Year                  0
일자Month                 0
일자Week                  0
일자Day                   0
일자Dayofweek             0
일자Dayofyear             0
일자Is_month_end          0
일자Is_month_start        0
일자Is_quarter_end        0
일자Is_quarter_start      0
일자Is_year_end           0
일자Is_year_start         0
일자Elapsed               0
net_work_num            0
net_office_num          0
off_office_num          0
net_office_ratio        0
off_office_ratio        0
overtime_work_ratio     0
dinner_isnull           0
work_home_ratio         0
dtype: int64

In [18]:
test = df_raw[df_raw['중식계'].isna()]
test.shape

(50, 30)

In [19]:
df_raw = df_raw[~(df_raw['중식계'].isna())]

In [51]:
df_raw = df_raw.drop(columns = '일자')

In [67]:
# 중식용이랑 석식용으로 데이터 만들기
df1_trn, y1_trn, nas_1 = proc_df(df_raw.drop(columns = ['석식계']), '중식계')
df2_trn, y2_trn, nas_2 = proc_df(df_raw[df_raw['dinner_isnull'] == False].drop(columns = ['중식계']), '석식계') # test 데이터에서도 중식계 feature를 알 수 없으므로 맞춰준다.

print(df1_trn.columns)
print(df2_trn.columns)

print(nas_1)
print(nas_2)
# 결측치는 없는 것으로...

# train valid split할 지점 확인 (8대 2으로 나눌 것임)
print(df1_trn.shape)
print(df1_trn.shape[0]*.8)
print(df1_trn.shape[0]*.2)

print(df2_trn.shape)
print(df2_trn.shape[0]*.8)
print(df2_trn.shape[0]*.2)

Index(['요일', '본사정원수', '본사휴가자수', '본사출장자수', '본사시간외근무명령서승인건수', '현본사소속재택근무자수',
       '일자Year', '일자Month', '일자Week', '일자Day', '일자Dayofweek', '일자Dayofyear',
       '일자Is_month_end', '일자Is_month_start', '일자Is_quarter_end',
       '일자Is_quarter_start', '일자Is_year_end', '일자Is_year_start', '일자Elapsed',
       'net_work_num', 'net_office_num', 'off_office_num', 'net_office_ratio',
       'off_office_ratio', 'overtime_work_ratio', 'dinner_isnull',
       'work_home_ratio'],
      dtype='object')
Index(['요일', '본사정원수', '본사휴가자수', '본사출장자수', '본사시간외근무명령서승인건수', '현본사소속재택근무자수',
       '일자Year', '일자Month', '일자Week', '일자Day', '일자Dayofweek', '일자Dayofyear',
       '일자Is_month_end', '일자Is_month_start', '일자Is_quarter_end',
       '일자Is_quarter_start', '일자Is_year_end', '일자Is_year_start', '일자Elapsed',
       'net_work_num', 'net_office_num', 'off_office_num', 'net_office_ratio',
       'off_office_ratio', 'overtime_work_ratio', 'dinner_isnull',
       'work_home_ratio'],
      dtype='object')
{}
{}
(1205, 27)
964

In [55]:
df1_trn.iloc[0]

요일                                4
본사정원수                          2601
본사휴가자수                           50
본사출장자수                          150
본사시간외근무명령서승인건수                  238
현본사소속재택근무자수                     0.0
일자Year                         2016
일자Month                           2
일자Week                            5
일자Day                             1
일자Dayofweek                       0
일자Dayofyear                      32
일자Is_month_end                False
일자Is_month_start               True
일자Is_quarter_end              False
일자Is_quarter_start            False
일자Is_year_end                 False
일자Is_year_start               False
일자Elapsed              1454284800.0
net_work_num                   2401
net_office_num               2401.0
off_office_num                200.0
net_office_ratio           0.923106
off_office_ratio           0.076894
overtime_work_ratio        0.099125
dinner_isnull                 False
work_home_ratio                 0.0
Name: 0, dtype: object

In [68]:
def split_vals(a,n): return a[:n], a[n:]   # 시간순으로 split

n_valid = 241
n_trn = len(df1_trn)-n_valid
X1_train, X1_valid = split_vals(df1_trn, n_trn)
y1_train, y1_valid = split_vals(y1_trn, n_trn)
#raw1_train, raw1_valid = split_vals(df_raw, n_trn)

# n_trn은 동일하니 생략
n_valid2 = 234
n_trn2= len(df2_trn) - n_valid2
X2_train, X2_valid = split_vals(df2_trn, n_trn2)
y2_train, y2_valid = split_vals(y2_trn, n_trn2)

#raw2_train, raw2_valid = split_vals(df_raw, n_trn)

print(X1_train.shape, len(y1_train))
print(X2_train.shape, len(y2_train))

(964, 27) 964
(934, 27) 934


In [108]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor

params = {
    'learning_rate': [0.0, 0.1, 0.09, 0.089, 0.08],
    'boosting_type': ['gbtree', 'gblinear', 'dart'],
    
}

lunch_r = XGBRegressor(objective='reg:squarederror')
dinner_r = XGBRegressor(objective='reg:squarederror')

lunch_model = GridSearchCV(lunch_r, params,n_jobs=-1, scoring='neg_mean_absolute_error')
dinner_model = GridSearchCV(dinner_r, params,n_jobs=-1, scoring='neg_mean_absolute_error')


%time lunch_model.fit(X1_train, y1_train)
print_score(lunch_model, X1_train, y1_train, X1_valid, y1_valid)
print('==========================================================')
print(lunch_model.best_score_, lunch_model.best_params_)
lunch_model = lunch_model.best_estimator_

# del m1
# gc.collect()
# # oob score : out of bag (무작위 샘플링 후 샘플링에 사용되지 않은 데이터들을 또 하나의 테스트 및 평가 대상으로 잡고 score를 내는 것)
# m1 = RandomForestRegressor(n_estimators=10, criterion = 'mae', n_jobs=-1, oob_score=True) # fitting결과를 보고 parameter 조정가능
# %time m1.fit(X1_train, y1_train)
# print_score(m1, X1_train, y1_train, X1_valid, y1_valid)

[18:17:12] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "boosting_type" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


Wall time: 23.5 s
[23.84066636334811, 204.00574642197225, -23.84066636334811, -204.00574642197225]
-73.36581435360027 {'boosting_type': 'gbtree', 'learning_rate': 0.089}


In [82]:
del m2
gc.collect()
m2 = RandomForestRegressor(n_estimators=40, criterion = 'mae', n_jobs=-1, oob_score=True)
%time m2.fit(X2_train, y2_train)
print_score(m2, X2_train, y2_train, X2_valid, y2_valid)

Wall time: 1.89 s
[17.616434689507496, 69.86079059829059, 0.9373185513266395, 0.557279965444675, 0.5795094120566686]


In [109]:
%time dinner_model.fit(X2_train, y2_train)
print_score(dinner_model, X2_train, y2_train, X2_valid, y2_valid)
print('==========================================================')
print(dinner_model.best_score_, dinner_model.best_params_)
dinner_model = dinner_model.best_estimator_

[18:17:48] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "boosting_type" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


Wall time: 19.5 s
[16.895824224075874, 84.79254815517328, -16.895824224075874, -84.79254815517328]
-56.251325294223065 {'boosting_type': 'gbtree', 'learning_rate': 0.08}


In [85]:
test = test.drop(columns = '일자')
test_X1, test_y1, nas_1 = proc_df(test, '중식계')
test_X2, test_y2, nas_2 = proc_df(test.drop(columns = ['중식계']), '석식계')

test.head(1)

,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,중식계,석식계,일자Year,일자Month,...,일자Is_year_start,일자Elapsed,net_work_num,net_office_num,off_office_num,net_office_ratio,off_office_ratio,overtime_work_ratio,dinner_isnull,work_home_ratio
0,수,2983,88,182,5,358.0,NaN,NaN,2021,1,...,False,1.611706e+09,2713,2355.0,628.0,0.789474,0.210526,0.001843,False,0.131957


In [88]:
test_X1.isna().sum()

요일                      0
본사정원수                   0
본사휴가자수                  0
본사출장자수                  0
본사시간외근무명령서승인건수          0
현본사소속재택근무자수             0
석식계                    50
일자Year                  0
일자Month                 0
일자Week                  0
일자Day                   0
일자Dayofweek             0
일자Dayofyear             0
일자Is_month_end          0
일자Is_month_start        0
일자Is_quarter_end        0
일자Is_quarter_start      0
일자Is_year_end           0
일자Is_year_start         0
일자Elapsed               0
net_work_num            0
net_office_num          0
off_office_num          0
net_office_ratio        0
off_office_ratio        0
overtime_work_ratio     0
dinner_isnull           0
work_home_ratio         0
석식계_na                  0
dtype: int64

In [90]:
test_X1 = test_X1.drop(columns = '석식계')

In [95]:
[i for i in X2_train.columns if i not in test_X2.columns]

[]

In [96]:
[i for i in test_X2.columns if i not in X2_train.columns]

[]

In [94]:
test_X1 = test_X1.drop(columns = '석식계_na')

In [111]:
# pred1 = m1.predict(test_X1)
# pred2 = m2.predict(test_X2)
pred1 = lunch_model.predict(test_X1)
pred2 = dinner_model.predict(test_X2)

submission['중식계'] = pred1
submission['석식계'] = pred2

submission

,일자,중식계,석식계
0,2021-01-27,638.085815,233.673782
1,2021-01-28,673.313477,379.574158
2,2021-01-29,542.320251,297.140442
3,2021-02-01,937.502502,380.041046
4,2021-02-02,732.198242,366.699646
5,2021-02-03,678.155762,306.385529
6,2021-02-04,809.909729,400.751434
7,2021-02-05,579.508972,341.973175
8,2021-02-08,988.700806,487.629059
9,2021-02-09,678.877686,415.837341


In [112]:
#submission.to_csv(f'{path}submission_210722_before_round.csv', index=False)
submission['중식계'] = submission['중식계'].apply(lambda x: round(x))
submission['석식계'] = submission['석식계'].apply(lambda x: round(x))


submission.to_csv(f'{path}submission_210722_gridsearchCV.csv', index=False)
submission

,일자,중식계,석식계
0,2021-01-27,638,234
1,2021-01-28,673,380
2,2021-01-29,542,297
3,2021-02-01,938,380
4,2021-02-02,732,367
5,2021-02-03,678,306
6,2021-02-04,810,401
7,2021-02-05,580,342
8,2021-02-08,989,488
9,2021-02-09,679,416


In [116]:
def rf_feat_importance(m, df):
    return pd.DataFrame({'cols':df.columns, 'imp':m.feature_importances_}
                       ).sort_values('imp', ascending=False)
fi = rf_feat_importance(m1, test_X1); 
display(fi[:10])
fi1_1 = rf_feat_importance(lunch_model, test_X1); fi1_1[:10]

,cols,imp
10,일자Dayofweek,0.327582
21,off_office_num,0.185369
3,본사출장자수,0.058987
18,일자Elapsed,0.051906
11,일자Dayofyear,0.043992
2,본사휴가자수,0.040935
22,net_office_ratio,0.038484
24,overtime_work_ratio,0.037250
9,일자Day,0.036924
4,본사시간외근무명령서승인건수,0.034019


,cols,imp
10,일자Dayofweek,0.793883
21,off_office_num,0.075098
22,net_office_ratio,0.024563
24,overtime_work_ratio,0.018486
3,본사출장자수,0.015866
4,본사시간외근무명령서승인건수,0.011278
2,본사휴가자수,0.009185
11,일자Dayofyear,0.008527
25,dinner_isnull,0.007306
7,일자Month,0.005877


In [115]:
fi2 = rf_feat_importance(m2, test_X2)
display(fi2[:10])
fi2_1 = rf_feat_importance(dinner_model, test_X2); fi2_1[:10]

,cols,imp
4,본사시간외근무명령서승인건수,0.148017
18,일자Elapsed,0.140549
24,overtime_work_ratio,0.119406
3,본사출장자수,0.089484
11,일자Dayofyear,0.074116
9,일자Day,0.057963
21,off_office_num,0.052825
22,net_office_ratio,0.051121
2,본사휴가자수,0.049157
23,off_office_ratio,0.041723


,cols,imp
24,overtime_work_ratio,0.254121
4,본사시간외근무명령서승인건수,0.149746
22,net_office_ratio,0.078639
18,일자Elapsed,0.075536
11,일자Dayofyear,0.074024
21,off_office_num,0.052307
10,일자Dayofweek,0.052161
2,본사휴가자수,0.049074
3,본사출장자수,0.040614
7,일자Month,0.040275
